# Smoking Detection using CNN

by Florian Rieser, June 2024

## 1. Project goal/Motivation

The No Smoking signs at gas stations are essential for safety, not just regulatory requirements. The main hazard at these locations is invisible: gasoline vapors. Using a FLIR Optical Gas Imaging camera, these unseen vapors become visible as a highly flammable black cloud that could ignite with minimal provocation. There are various methods to detect smoking, such as smoke detectors, gas sensors (e.g., carbon monoxide and VOC sensors), particulate matter sensors, thermal cameras, acoustic sensors, and chemical sensors. Although effective, these methods can be expensive to implement.

This project aims to provide a more affordable solution by utilizing artificial intelligence. By training an artificial neural network to recognize smoking behavior based on pictures of individuals, we can develop an alert system that ensures safety without the need for costly specialized equipment. This model can then be combined with an object detection system to identify individuals and detect smoking, triggering necessary alerts. By leveraging the existing surveillance cameras typically found at gas stations, this AI-based approach offers a cost-effective and efficient way to prevent accidents caused by smoking in restricted areas.

## 2. Data Collection

For this project, we utilized a dataset from Kaggle (https://www.kaggle.com/datasets/sujaykapadnis/smoking), specifically designed for smoker detection. The dataset consists of 1,120 images, equally divided into two classes: 560 images of smokers and 560 images of non-smokers. These images were collected by searching various keywords across multiple search engines, including terms such as cigarette smoking, smoker, person, coughing, taking inhaler, person on the phone, and drinking water. This approach ensured a diverse set of images for each category.

To enhance the model's training and introduce a degree of inter-class confusion, the dataset includes versatile images within both classes. The Smoking class features images of individuals smoking from different angles and in various gestures. In contrast, the NotSmoking class contains images of non-smokers performing actions that could resemble smoking, such as drinking water, using an inhaler, holding a mobile phone, or coughing. This careful selection improves the model's ability to accurately differentiate between smokers and non-smokers.

All images were preprocessed and resized to a resolution of 250×250 pixels. The dataset was split, with 80% used for training and validation and the remaining 20% for testing. This dataset is a valuable resource for developing deep learning algorithms for automated smoker detection, contributing to environmental sustainability and enhancing surveillance in smart cities.

Citation: Khan, Ali (2022), “Smoker Detection Dataset”, Mendeley Data, V1, doi: 10.17632/j45dj8bgfc.1

## 3. Training/Fine-tuning of a self-developed model

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import re

In [ ]:
# Define the paths to the training and validation directories
train_dir = 'training_images'
validation_dir = 'validation_images'

# Define the image size and batch size
image_size = (250, 250) # All images will be resized to 250x250
batch_size = 32 # Number of images to process at a time
num_epochs = 500 # Number of epochs to train the model

In [ ]:
# Function to load images from directory
def load_images_from_directory(directory, target_size=(250, 250)):
    images = []
    labels = []
    class_names = sorted(os.listdir(directory))  # Get sorted list of class names
    label_to_index = {class_name: i for i, class_name in enumerate(class_names)}  # Map class name to numerical label
    for class_name in class_names:
        class_dir = os.path.join(directory, class_name)
        for filename in os.listdir(class_dir):
            image_path = os.path.join(class_dir, filename)
            image = Image.open(image_path)
            image = image.resize(target_size)
            image = np.array(image) / 255.0  # Normalize pixel values
            images.append(image)
            labels.append(label_to_index[class_name])  # Append numerical label
    return np.array(images), np.array(labels), class_names

# Load and preprocess the training, validation, and testing datasets
train_images, train_labels, train_class_labels = load_images_from_directory(train_dir)
validation_images, validation_labels, validation_class_labels = load_images_from_directory(validation_dir)

In [ ]:
# Data augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal_and_vertical'),
    tf.keras.layers.RandomRotation(0.2),
])

In [ ]:
# Define the CNN model for binary classification
model = tf.keras.Sequential([
    tf.keras.Input(shape=(250, 250, 3)),  # Define input shape
    data_augmentation, 
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'), 
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'), 
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'), 
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'), 
    tf.keras.layers.MaxPooling2D((2, 2)), 
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(256, activation='relu'), 
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu'), 
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'), 
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32, activation='relu'), 
    tf.keras.layers.Dense(1, activation='sigmoid') # Use 'sigmoid' for binary classification
])

In [ ]:
# Define the optimizer with a specific learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001) 

# Compile the model with the custom optimizer
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              optimizer=optimizer,
              metrics=['accuracy'])

In [ ]:
# Train the model and save the training history
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

history = model.fit(
    train_images, train_labels,
    batch_size=batch_size,
    epochs=num_epochs,
    validation_data=(validation_images, validation_labels),
    callbacks=[early_stopping_cb]
)

In [ ]:
# Plot training & validation accuracy and loss values
plt.figure(figsize=(12, 5))

# Plot training & validation accuracy values
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper left')

plt.show()

In [ ]:
# Save the model with the incremented version number
def get_latest_version_number():
    if not os.path.exists('models'):
        return 0
    model_files = os.listdir('models')
    version_numbers = [int(re.search(r'_v(\d+)\.keras', filename).group(1)) for filename in model_files if re.search(r'_v(\d+)\.keras', filename)]
    if version_numbers:
        return max(version_numbers)
    else:
        return 0

def save_model_with_version(model):
    if not os.path.exists('models'):
        os.makedirs('models')  # Create the 'models' folder if it does not exist
    latest_version = get_latest_version_number()
    version = latest_version + 1
    model_path = f'models/smokingdetector_v{version}.keras'
    model.save(model_path)
    print(f"Model saved as {model_path}")

save_model_with_version(model)

## 4. Interpretation and validation of results & model performance measure

In [ ]:
# Evaluate the model on the validation set
latest_version = get_latest_version_number()
model_path = f'models/smokingdetector_v{latest_version}.keras'
model = tf.keras.models.load_model(model_path)

loss, accuracy = model.evaluate(validation_images, validation_labels)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

# Predict the labels for the validation set
predictions = model.predict(validation_images)
predicted_labels = (predictions > 0.5).astype(int).flatten()

# Calculate and print AUC-ROC
auc_roc = roc_auc_score(validation_labels, predictions)
print(f"AUC-ROC: {auc_roc}")

# Calculate the confusion matrix
conf_matrix = confusion_matrix(validation_labels, predicted_labels)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['not smoking', 'smoking'], yticklabels=['not smoking', 'smoking'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

# Calculate True Positives (TP), False Positives (FP), False Negatives (FN)
TP = np.sum((predicted_labels == 1) & (validation_labels == 1))
FP = np.sum((predicted_labels == 1) & (validation_labels == 0))
FN = np.sum((predicted_labels == 0) & (validation_labels == 1))

# Calculate Precision and Recall
precision = TP / (TP + FP)
recall = TP / (TP + FN)

print(f"Precision: {precision}")
print(f"Recall: {recall}")

## 5. Apply the model to your own images

In [ ]:
# Evaluate the model on the validation set
latest_version = get_latest_version_number()
model_path = f'models/smokingdetector_v{latest_version}.keras'
model = tf.keras.models.load_model(model_path)

# Load and preprocess the image you want to predict
image_path = "path_to_your_image.jpg"  # Replace "path_to_your_image.jpg" with the path to your image
image = Image.open(image_path)
image = image.resize((250, 250))  # Resize the image to match the input size of the model
image = np.array(image) / 255.0  # Normalize pixel values

# Make predictions
predictions = model.predict(np.expand_dims(image, axis=0))

# Interpret predictions
if predictions[0][0] > 0.5:
    print("The model predicts that the image contains smoking behavior.")
else:
    print("The model predicts that the image does not contain smoking behavior.")